In [1]:
%matplotlib inline
import os
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
import torchvision.transforms as tt
import numpy as np
from tqdm import tqdm

In [2]:
class ToDeviceLoader:
    def __init__(self,data,device):
        self.data = data
        self.device = device
        
    def __iter__(self):
        for batch in self.data:
            yield to_device(batch,self.device)
            
    def __len__(self):
        return len(self.data)
    
def to_device(data,device):
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking=True)

def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

In [3]:
# HYPER_PARAMETER
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device_ids = [0]
NUM_CLASSES = 3
DATA_DIR = "/home/qianq/data/lung_png"
device = get_device()
BATCH_SIZE = 1
CROP_SIZE = (512, 512) # 在代码不修改的情况 crop 调整为最小值
LEARNING_RATE = 5e-5
NUM_EPOCHS = 2
SEG_COLORMAP = [
    [0, 0, 0], 
    [255, 255, 255], 
    [255, 106, 106], 
]
SEG_CLASSES = [
    'background', 
    'lung_left', 
    'lung_right'
]

In [4]:

# def fname_to_ids(fname):
#     """
#     return ct_id and ct_slice_id
#     volume-001-058.png -> 1, 58
#     """
#     seg_list = fname.split('-')
#     return int(seg_list[1]), int(seg_list[2].replace('.png', ''))

# mode = torchvision.io.image.ImageReadMode.RGB
# data_dir = DATA_DIR
# source_list = os.listdir(os.path.join(DATA_DIR, 'source'))
# for fname in source_list[137:]:
#     source = torchvision.io.read_image(os.path.join(data_dir, 'source', fname), mode)
#     ct_id, slice_id = fname_to_ids(fname)
#     target_fname_lung_left = "seg-lung-left-%03d-%03d.png" % (ct_id, slice_id)
#     target_fname_lung_right = "seg-lung-right-%03d-%03d.png" % (ct_id, slice_id)
#     label_left = torchvision.io.read_image(os.path.join(data_dir, 'target', target_fname_lung_left), mode)
#     label_right = torchvision.io.read_image(os.path.join(data_dir, 'target', target_fname_lung_right), mode)
#     if (
#         source.shape != torch.Size([3, 512, 512])
#         or label_left.shape != torch.Size([3, 512, 512])
#         or label_right.shape != torch.Size([3, 512, 512])
#     ):
#         print(fname)
#         print(source.shape)
#         print(label_left.shape)
#         print(label_right.shape)

In [5]:
class FullConvolutionalNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_net = nn.Sequential(
            *list(
                torchvision.models.resnet18(weights="IMAGENET1K_V1").children()
            )[:-2]
        )
        self.final_conv = nn.Conv2d(512, NUM_CLASSES, kernel_size=1)
        self.transpose_conv = nn.ConvTranspose2d(
            NUM_CLASSES, 
            NUM_CLASSES,
            kernel_size=64, 
            padding=16, 
            stride=32,
        )
        W = self.bilinear_kernel(NUM_CLASSES, NUM_CLASSES, 64)
        self.transpose_conv.requires_grad_(False)
        self.transpose_conv.weight.copy_(W)
        self.transpose_conv.requires_grad_(True)
        self._model = nn.Sequential(
            self.pretrained_net,
            self.final_conv,
            self.transpose_conv,
        )
    
    def bilinear_kernel(self, in_channels, out_channels, kernel_size):
        factor = (kernel_size + 1) // 2
        if kernel_size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = (torch.arange(kernel_size).reshape(-1, 1),
            torch.arange(kernel_size).reshape(1, -1))
        filt = (1 - torch.abs(og[0] - center) / factor) * \
            (1 - torch.abs(og[1] - center) / factor)
        weight = torch.zeros((in_channels, out_channels,
                            kernel_size, kernel_size))
        weight[range(in_channels), range(out_channels), :, :] = filt
        return weight


    def forward(self, x):
        return self._model(x)

In [6]:
class LungSegDataset(torch.utils.data.Dataset):
    """A customized dataset to load the LitsLiverDataset`"""

    normalize_transform = tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    resize_transform = tt.Resize(CROP_SIZE)
    

    def __init__(self, is_train, crop_size, data_dir, object='lung'):
        """
        object[choice]:
            lung_left
            lung_right
            all
        """
        self.transform = self.normalize_transform
        self.crop_size = crop_size
        self.is_train = is_train
        self.data_dir = data_dir
        self.object = object

        self.source_images = os.listdir(os.path.join(data_dir, 'source'))
        self.train_source_list = []
        self.test_source_list = []
        for fname in self.source_images:
            ct_id, slice_id = self.fname_to_ids(fname)
            ct_id = int(ct_id)
            if ct_id <= 100:
                self.train_source_list.append(fname)
            elif ct_id > 100:
                self.test_source_list.append(fname)
        self.colormap2label = self.get_colormap2label()

    @classmethod
    def normalize_image(cls, img):
        return cls.normalize_transform(img.float() / 255)

    def filter(self, imgs):
        return [img for img in imgs if (
            img.shape[1] >= self.crop_size[0] and
            img.shape[2] >= self.crop_size[1])]

    def __getitem__(self, idx):
        feature, label = self.read_single_image(
            object=self.object, 
            idx=idx,
        )
        feature = self.normalize_image(feature)
        feature, label = self.rand_crop(feature, label,*self.crop_size)
        return (feature, self.label_indices(label, self.colormap2label))

    def __len__(self):
        if self.is_train:
            return len(self.train_source_list)
        else:
            return len(self.test_source_list)

    def read_single_image(self, idx, object='lung'):
        mode = torchvision.io.image.ImageReadMode.RGB
        if self.is_train:
            source_images_list = self.train_source_list
        else:
            source_images_list = self.test_source_list
        fname = source_images_list[idx]
        ct_id, slice_id = self.fname_to_ids(fname)
        ct_id = int(ct_id)
        slice_id = int(slice_id)
        feature = torchvision.io.read_image(
            os.path.join(self.data_dir, 'source', fname),
            mode,
        )
        target_fname_lung_left = "seg-lung-left-%03d-%03d.png" % (ct_id, slice_id)
        target_fname_lung_right = "seg-lung-right-%03d-%03d.png" % (ct_id, slice_id)

        if object in ['lung_left','lung_right']:
            target_fname = target_fname_lung_left if object == 'lung_left' else target_fname_lung_right
            label = torchvision.io.read_image(os.path.join(self.data_dir, 'target', target_fname), mode)
        else:
            # both lung
            label_lung_left = torchvision.io.read_image(os.path.join(self.data_dir, 'target', target_fname_lung_left), mode)
            label_lung_right = torchvision.io.read_image(os.path.join(self.data_dir, 'target', target_fname_lung_right), mode)
            # print(label_lung_left.max())
            # print(label_lung_right.max())
            label_idx = label_lung_right[0] == 255
            label_lung_right[1][label_idx] = 106
            label_lung_right[2][label_idx] = 106
            label = label_lung_left + label_lung_right
        # print(feature.shape, label.shape)
        feature = self.resize_transform(feature)
        label = self.resize_transform(label)
        return feature, label

    # @classmethod
    # def read_images(cls, data_dir, object='liver', is_train=True):
    #     """
    #     Read all Lits feature and liver-label images on the memory
    #     [warning] it may cause machine crash
    #     object[choice]:
    #         - liver
    #         - tumor
    #     is_train:
    #         - True: first 100 ct samples
    #         - False: last 30 ct samples
    #     """
    #     mode = torchvision.io.image.ImageReadMode.RGB
    #     features, labels = [], []
    #     images = os.listdir(os.path.join(data_dir, 'source'))
    #     pbar = tqdm(total=len(images))
    #     for i, fname in enumerate(images):
    #         ct_id, slice_id = cls.fname_to_ids(fname)
    #         ct_id = int(ct_id)
    #         slice_id = int(slice_id)
    #         if is_train and ct_id > 100:
    #             continue
    #         elif (not is_train) and ct_id <= 100:
    #             continue
    #         features.append(torchvision.io.read_image(
    #             os.path.join(data_dir, 'source', fname), mode)
    #         )
    #         if object == 'liver':
    #             target_fname = "seg-liver-%03d-%03d.png" % (ct_id, slice_id)
    #         else:
    #             target_fname = "seg-tumor-%03d-%03d.png" % (ct_id, slice_id)
    #         label = torchvision.io.read_image(os.path.join(data_dir, 'target', target_fname), mode)
    #         labels.append(label)
    #         pbar.update(1)
    #     return features, labels

    @classmethod
    def fname_to_ids(cls, fname):
        """
        return ct_id and ct_slice_id
        volume-001-058.png -> 1, 58
        """
        seg_list = fname.split('-')
        return seg_list[1], seg_list[2].replace('.png', '')

    def get_colormap2label(self):
        """Build the mapping from RGB to class indices for labels.

        Defined in :numref:`sec_semantic_segmentation`"""
        colormap2label = torch.zeros(256 ** 3, dtype=torch.long)
        for i, colormap in enumerate(SEG_COLORMAP):
            colormap2label[
                (colormap[0] * 256 + colormap[1]) * 256 + colormap[2]] = i
        return colormap2label

    def rand_crop(self, feature, label, height, width):
        """Randomly crop both feature and label images.

        Defined in :numref:`sec_semantic_segmentation`"""
        rect = torchvision.transforms.RandomCrop.get_params(
            feature, (height, width))
        feature = torchvision.transforms.functional.crop(feature, *rect)
        label = torchvision.transforms.functional.crop(label, *rect)
        return feature, label

    def label_indices(self, colormap, colormap2label):
        """Map any RGB values in labels to their class indices.

        Defined in :numref:`sec_semantic_segmentation`"""
        colormap = colormap.permute(1, 2, 0).numpy().astype('int32')
        idx = ((colormap[:, :, 0] * 256 + colormap[:, :, 1]) * 256
               + colormap[:, :, 2])
        return colormap2label[idx]

    @classmethod
    def label2image(cls, pred, device):
        colormap = torch.tensor(SEG_COLORMAP, device=device)
        X = pred.long()
        return colormap[X, :]

In [7]:
# ds = LungSegDataset(True, (512,512), DATA_DIR, 'lung')
# source, target = ds[76]

In [8]:
class AverageMeter:
    ''' Computes and stores the average and current value '''
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val: float, n: int = 1) -> None:
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        

def get_dataloader_workers():
    """Use 4 processes to read the data."""
    return 4

def load_data_lits(batch_size, crop_size):
    """Load the LITS liver semantic segmentation dataset.

    Defined in :numref:`sec_semantic_segmentation`"""
    data_dir = DATA_DIR
    num_workers = get_dataloader_workers()
    
    train_iter = torch.utils.data.DataLoader(
        LungSegDataset(True, crop_size, data_dir, object='all'), batch_size,
        shuffle=True, drop_last=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(
        LungSegDataset(False, crop_size, data_dir, object='all'), batch_size,
        drop_last=True, num_workers=num_workers)
    return train_iter, test_iter

# def loss(inputs, targets):
#     return F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)

reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)


def pixel_accuracy(y_hat, y):
    """Compute the number of correct predictions.[each pixel]

    Defined in :numref:`sec_softmax_scratch`"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    size = 1
    for s in cmp.shape:
        size *= s
    return float(cmp.sum()/size)


def calc_iou(pred, target, n_classes = 2):
    #n_classes ：the number of classes in your dataset
    ious = []
    pred = argmax(pred, axis=1)
    pred = pred.view(-1)
    target = target.view(-1)

    # Ignore IoU for background class ("0")
    for cls in range(1, n_classes):  # This goes from 1:n_classes-1 -> class "0" is ignored
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds[target_inds]).long().sum().data.cpu().item()  # Cast to long to prevent overflows
        union = pred_inds.long().sum().data.cpu().item() + target_inds.long().sum().data.cpu().item() - intersection
        # if union == 0:
        #     ious.append(float('nan'))  # If there is no ground truth, do not include in evaluation
        # else:
        #     ious.append(float(intersection) / float(max(union, 1)))
        if union != 0:
            ious.append([float(intersection), float(max(union, 1))])
        
    # return np.array(ious)
    return ious

In [9]:
train_iter, test_iter = load_data_lits(BATCH_SIZE, CROP_SIZE)
train_dl = ToDeviceLoader(train_iter, device)
test_dl = ToDeviceLoader(test_iter, device)

In [10]:
model = FullConvolutionalNet()
model = model.to(device)
# model = torch.nn.DataParallel(model, device_ids=device_ids)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, LEARNING_RATE, eta_min=0)
lr_scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, LEARNING_RATE, total_iters=NUM_EPOCHS)
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=LEARNING_RATE, epochs=NUM_EPOCHS, steps_per_epoch=len(train_dl)+1)

In [11]:
print(device)
print(torch.cuda.is_available())

cuda
True


In [12]:
def train(train_iter, model, criterion, optimizer, epoch, total_epochs, lr_scheduler):
    loss_metric = AverageMeter()
    acc_metric = AverageMeter()
    num_steps = len(train_iter)
    lr_rate = lr_scheduler.get_last_lr()
    pbar = tqdm(total=num_steps)
    pbar.set_description(f"[Train] EPOCH [{epoch}/{total_epochs}], lr=[{lr_rate}]:")
    for i, (x, y) in enumerate(train_iter):
        batch_cnt = x.shape[0]
        model.train()
        optimizer.zero_grad()
        pred = model(x)
        # print(x.shape)
        # print(y.shape)
        # print(pred.shape)
        l = criterion(pred, y)
        l.sum().backward()
        train_loss_sum = l.sum()
        train_acc_sum = pixel_accuracy(pred, y)
        optimizer.step()
        lr_scheduler.step()
        loss_metric.update(float(train_loss_sum), n=batch_cnt)
        acc_metric.update(train_acc_sum, n=1)
        pbar.update(1)
        pbar.set_postfix(loss=loss_metric.avg, acc=acc_metric.avg)
    return loss_metric.avg, acc_metric.avg


def valid(test_iter, model, criterion, epoch, total_epochs, lr_scheduler):
    loss_metric = AverageMeter()
    acc_metric = AverageMeter()
    num_steps = len(test_iter)
    lr_rate = lr_scheduler.get_last_lr()
    pbar = tqdm(total=num_steps)
    pbar.set_description(f"[Valid] EPOCH [{epoch}/{total_epochs}], lr=[{lr_rate}]:")
    for i, (x, y) in enumerate(test_iter):
        batch_cnt = x.shape[0]
        model.eval()
        pred = model(x)
        l = criterion(pred, y)
        train_loss_sum = l.sum()
        train_acc_sum = pixel_accuracy(pred, y)
        loss_metric.update(float(train_loss_sum), n=batch_cnt)
        acc_metric.update(train_acc_sum, n=1)
        pbar.update(1)
        pbar.set_postfix(loss=loss_metric.avg, acc=acc_metric.avg)
    return loss_metric.avg, acc_metric.avg
    

In [13]:
history = []
for epoch in range(1, NUM_EPOCHS+1):
    print('-' * 50)
    lr_scheduler.get_last_lr()
    train_loss, train_acc = train(train_dl, model, criterion, optimizer, epoch, NUM_EPOCHS, lr_scheduler)
    valid_loss, valid_acc = valid(test_dl, model, criterion, epoch, NUM_EPOCHS, lr_scheduler)
    history.append([epoch, train_loss, train_acc, valid_loss, valid_acc])

--------------------------------------------------


[Train] EPOCH [1/2], lr=[[2.5e-09]]::   0%|          | 0/3427 [00:00<?, ?it/s]/home/qianq/mycodes/lits17-seg/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/qianq/mycodes/lits17-seg/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resiz

KeyboardInterrupt: 

[Train] EPOCH [1/2], lr=[[2.5e-09]]::  21%|██        | 727/3427 [00:29<00:45, 58.79it/s, acc=0.97, loss=0.0953]

In [ ]:
def show_imgs(imgs):
    row_cnt = len(imgs)
    col_cnt = len(imgs[0])
    plt.figure(figsize=(6,8)) #设置窗口大小
    plt.suptitle('lung seg') # 图片名称
    i = 0
    for (source, pred, target) in imgs:
        s_idx = i*col_cnt + 1
        p_idx = s_idx + 1
        t_idx = p_idx + 1
        plt.subplot(row_cnt, col_cnt, s_idx), plt.title('source')
        plt.imshow(source), plt.axis('off')
        plt.subplot(row_cnt, col_cnt, p_idx), plt.title('pred')
        plt.imshow(pred), plt.axis('off') #这里显示灰度图要加cmap
        plt.subplot(row_cnt, col_cnt, t_idx), plt.title('target')
        plt.imshow(target), plt.axis('off')
        i = i+1
    plt.show()

In [ ]:
ds = LungSegDataset(False, CROP_SIZE, DATA_DIR, object='all')
idx1 = ds.test_source_list.index("volume-120-191.png")
idx2 = ds.test_source_list.index("volume-129-228.png")
idx_list = [idx1, idx1+500, idx2, idx2+50]
n = 4
X = torch.zeros((n, 3, 512, 512))
Y = torch.zeros((n, 512, 512))

for i in range(n):
    X[i], Y[i] = ds[idx_list[i]]

y_hat = model(X).argmax(dim=1)
# print(X.shape)
imgs = []

for i in range(n):
    x = X[i].permute(1, 2, 0).to('cpu')
    pred = LungSegDataset.label2image(y_hat[i], device).to('cpu')
    y = LungSegDataset.label2image(Y[i], device).to('cpu')
    imgs.append([x, pred, y])

show_imgs(imgs)
# print(imgs[0].shape)